In [ ]:
import re                          # Regular Expressions
import pandas as pd                # DataFrames & Manipulation

In [ ]:
train_input = "../data/recipes.tsv.bz2"

# keep empty strings (http://pandas-docs.github.io/pandas-docs-travis/io.html#na-values)
train = pd.read_csv(train_input, delimiter="\t", quoting=3, encoding="utf-8", keep_default_na=False)

print "loaded %d documents." % len(train)

In [ ]:
train[['title', 'instructions']].head()